In [1]:
import json
import faiss
import numpy as np
import torch
from pathlib import Path
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

In [2]:
# --- КОНФИГ ---
CHUNKS_FILE = Path('../data/processed/chunks.jsonl')
INDEX_DIR = Path('../data/processed/faiss_index')
INDEX_DIR.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"
BATCH_SIZE = 4

In [3]:
model = SentenceTransformer(MODEL_NAME, trust_remote_code=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [4]:
texts = []
metadata = []
ids = []

with open(CHUNKS_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        texts.append(item['text'])
        metadata.append({
            "arxiv_id": item['id'],
            "source": item['source'],
            "meta": item['metadata']
        })
        ids.append(item['id'])

print(f"Всего чанков: {len(texts)}")

Всего чанков: 4179


In [5]:
embeddings = model.encode(
    texts, 
    batch_size=BATCH_SIZE, 
    show_progress_bar=True, 
    convert_to_numpy=True, 
    normalize_embeddings=True
)

Batches:   0%|          | 0/1045 [00:00<?, ?it/s]

In [6]:
print("Строим индекс FAISS...")
dimension = embeddings.shape[1]

index = faiss.IndexFlatIP(dimension) 
index.add(embeddings)

# 4. Сохраняем всё на диск
print("Сохраняем на диск...")

# Сохраняем сам векторный индекс
faiss.write_index(index, str(INDEX_DIR / "index.faiss"))

# Сохраняем метаданные (порядок совпадает с индексом FAISS)
with open(INDEX_DIR / "metadata.json", 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False)
    
print(f"Готово! Индекс сохранен в {INDEX_DIR}")
print(f"Размер индекса: {index.ntotal} векторов.")

Строим индекс FAISS...
Сохраняем на диск...
Готово! Индекс сохранен в ..\data\processed\faiss_index
Размер индекса: 4179 векторов.
